In [25]:
# Importing the common libraries
import numpy as np
import pandas as pd
import geopandas as gpd
import re

# TensorFlow and Scikit Learn libraries
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, InputLayer
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import RootMeanSquaredError, MeanAbsoluteError

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

In [26]:
# Create a function that imports the data and returns 6 dataframes for each of the 6 models
def importData():
    # Importing the data
    df = pd.read_csv('../../Data/GNIP/GNIP_Cleaned.csv')
    cols = df.columns

    # Remove units from columns for easier processing
    codeCols = list(map(lambda x: re.sub(r'\(([^()]*)\)', '', x).strip(), cols))
    df.columns = codeCols

    # Load in the PrevailingWinds file for which the model will be split on
    
    return df, cols

In [27]:
# Main function to call all the other functions
def main():
    # Importing the data
    df, cols = importData()
    return df

In [28]:
df = main()
df.head()

,Lat,Lon,Date,Alt,Precip,Temp,O18,H2
0,-75.583333,-20.566667,1965-07-15 00:00:00+02:00,30.0,28.0,-27.6,-22.12,-188.8
1,-75.583333,-20.566667,1965-08-15 00:00:00+02:00,30.0,37.0,-30.0,-24.31,-195.7
2,-75.583333,-20.566667,1965-09-15 00:00:00+02:00,30.0,27.0,-26.0,-19.11,-155.7
3,-75.583333,-20.566667,1965-10-15 00:00:00+02:00,30.0,43.0,-20.1,-19.70,-166.7
4,-75.583333,-20.566667,1966-01-15 00:00:00+01:00,30.0,26.0,-5.4,-10.58,-99.2
